# Task 2
Нужно сгенерировать файл, содержащий 2000 48-битных случайных целых чисел, каждое число на отдельной строке.

Посчитать, какое суммарное количество простых множителей присутствует при факторизации всех чисел. Например, пусть всего два числа:

6 и 8. 6 = 2 \* 3, 8 = 2 \* 2 \* 2. Ответ 5.

Реализовать подсчет:
- простым последовательным алгоритмом
- многопоточно, с использованием примитивов синхронизации
- с помощью Akka (или аналога)
- c помощью RxJava (или аналога)


Измерить время выполнения для каждого случая. Использовать уровень параллельности в соответствии с числом ядер вашего CPU.



## Генерация данных
В файле создаются 2000 случайных 48-битных целых чисел 

In [1]:
num_int = 2000
file_name = "int48file"

In [2]:
from generator import create_file_with_int48

create_file_with_int48(num_int, file_name)

Функция факторизации

In [3]:
def calc_num_factor(n):
    ans = 0
    d = 2
    while d * d <= n:
        if n % d == 0:
            ans += 1
            n //= d
        else:
            d += 1
    if n > 1:
        ans += 1
    return ans

## Последовательная факторизация + своя функция факторизации

In [4]:
from generator import read_f
import numpy as np

In [5]:
%%time

# sum = 0
# array = read_f(file_name)
# for elem in array:
#     print(elem)
#     sum += calc_num_factor(elem)

Wall time: 0 ns


## Последовательная факторизация + sympy

In [6]:
from sympy.ntheory import factorint

In [7]:
def get_sum_factorization(num):
    return sum(factorint(num).values())

In [8]:
%%time

arr = read_f(file_name)
sum(np.vectorize(get_sum_factorization)(arr))

Wall time: 1.73 s


9018

## Многопоточная факторизация + sympy

In [9]:
def get_splited(f_n, num):
    return np.array_split(read_f(f_n), num)

In [10]:
from multiprocessing.dummy import Pool

num_process = 8

In [11]:
%%time

sptlited_arrays = get_splited(file_name, num_process)
sum(sum(Pool(num_process).map(np.vectorize(get_sum_factorization), sptlited_arrays)))

Wall time: 1.63 s


9018

## Многопроцессорная факторизация + sympy

In [12]:
from multiprocessing import Pool as pool_proc

In [13]:
%%time

# sptlited_arrays = get_splited(file_name, num_process)
# sum(sum(pool_proc(num_process).map(np.vectorize(get_sum_factorization), sptlited_arrays)))

Wall time: 0 ns


## Многопоточная факторизация Pykka

In [14]:
import pykka

In [15]:
class factoriser(pykka.ThreadingActor):
    def on_receive(self, array):
        return np.sum(np.vectorize(get_sum_factorization)(array))

In [16]:
%%time

sptlited_arrays = get_splited(file_name, num_process)
factoriser = factoriser.start()

result = np.sum([factoriser.ask(elem) for elem in sptlited_arrays])
pykka.ActorRegistry.stop_all()
result

Wall time: 1.71 s


9018

## Многопоточная факторизация RxPY

In [17]:
import rx
from rx import operators as ops

In [18]:
def output(result):
    print('%d' % result)

In [19]:
%%time

source = rx.of(*read_f(file_name))
source.pipe(
    ops.map(get_sum_factorization),
    ops.sum()
).subscribe(output)


9018
Wall time: 1.78 s


## Результаты
В результате данной работы получены следующие результаты:
- последовательная 1.73c
- многопоточная 1.63c
- многопоточная Pykka 1.71c
- многопоточная RxPY 1.78c

Во всех приведенных вариантах для факторизации использовалась функция из библиотеки SymPy. При использовании своей функции результат превосходил разумные временные рамки в сравнении с приведенными выше.
Кроме того, было выявленно странное поведение распараллеливания, основанного на многопроцессорности – также результат вычислялся несоизмеримо дольше.